# Azure ML v2 Cloud Inference Example

This notebook demonstrates how to deploy and use a machine learning model for cloud inference using Azure Machine Learning SDK v2.

## Overview

1. **Setup and Configuration**: Load required libraries and configure Azure ML workspace
2. **Model Training**: Create and train a sample model
3. **Model Registration**: Register the model in Azure ML
4. **Endpoint Creation**: Create a managed online endpoint
5. **Model Deployment**: Deploy the model to the endpoint
6. **Testing**: Test the deployed model with various input formats
7. **Cleanup**: Clean up resources when done

## 1. Setup and Configuration

In [ ]:
# Install required packages (run only if needed)
# !pip install azure-ai-ml azure-identity scikit-learn pandas numpy joblib python-dotenv

In [ ]:
import os
import sys
import json
import pandas as pd
import numpy as np
from dotenv import load_dotenv

# Add src directory to path
sys.path.append('../src')

from aml_cloud_inference import AMLCloudInference
from model.train_model import create_sample_data, train_model, save_model_and_data

# Load environment variables
load_dotenv()

print("✅ Libraries imported successfully!")

In [ ]:
# Configure Azure ML workspace
config = {
    "subscription_id": os.getenv("AZURE_SUBSCRIPTION_ID"),
    "resource_group": os.getenv("AZURE_RESOURCE_GROUP"),
    "workspace_name": os.getenv("AZURE_ML_WORKSPACE_NAME"),
}

# Check configuration
missing_vars = [k for k, v in config.items() if not v]
if missing_vars:
    print("❌ Missing environment variables:")
    for var in missing_vars:
        print(f"  - {var.upper()}")
    print("\nPlease set these in your .env file or environment.")
else:
    print("✅ Configuration loaded successfully!")
    print(f"Workspace: {config['workspace_name']}")
    print(f"Resource Group: {config['resource_group']}")

## 2. Model Training

In [ ]:
# Create sample data
print("📊 Creating sample data...")
X, y = create_sample_data(n_samples=1000, n_features=10)

print(f"Data shape: {X.shape}")
print(f"Target distribution: {np.bincount(y)}")
print("✅ Sample data created!")

In [ ]:
# Train the model
print("🤖 Training model...")
model = train_model(X, y)

# Save model and sample data
save_model_and_data(model, X, y, "../src/model")
print("✅ Model training completed!")

## 3. Azure ML Setup

In [ ]:
# Initialize Azure ML client
print("🔗 Connecting to Azure ML workspace...")
client = AMLCloudInference(**config)
print("✅ Connected to Azure ML workspace!")

In [ ]:
# List existing endpoints
endpoints = client.list_endpoints()
print(f"📋 Found {len(endpoints)} existing endpoints:")
for endpoint in endpoints:
    print(f"  - {endpoint['name']}: {endpoint['scoring_uri']}")

## 4. Model Registration and Deployment

In [ ]:
# Configuration for deployment
model_name = "notebook-sample-classifier"
endpoint_name = "notebook-classifier-endpoint"
deployment_name = "notebook-classifier-deployment"

print(f"Model name: {model_name}")
print(f"Endpoint name: {endpoint_name}")
print(f"Deployment name: {deployment_name}")

In [ ]:
# Register the model
print(f"📝 Registering model '{model_name}'...")
registered_model = client.register_model(
    model_name=model_name,
    model_path="../src/model",
    description="Sample RandomForest classifier from Jupyter notebook"
)
print(f"✅ Model registered with version: {registered_model.version}")

In [ ]:
# Create endpoint
print(f"🌐 Creating endpoint '{endpoint_name}'...")
endpoint = client.create_endpoint(
    endpoint_name=endpoint_name,
    description="Sample endpoint created from Jupyter notebook"
)
print(f"✅ Endpoint created: {endpoint.scoring_uri}")

In [ ]:
# Create deployment (this may take several minutes)
print(f"🚀 Creating deployment '{deployment_name}'...")
print("⏱️ This may take 5-10 minutes...")

deployment = client.create_deployment(
    endpoint_name=endpoint_name,
    deployment_name=deployment_name,
    model_name=model_name,
    model_version=registered_model.version,
    code_path="../src",
    scoring_script="score.py",
    instance_type="Standard_DS3_v2",
    instance_count=1
)
print("✅ Deployment created successfully!")

In [ ]:
# Set traffic to 100%
print("🔄 Setting traffic allocation to 100%...")
client.set_traffic(endpoint_name, deployment_name, 100)
print("✅ Traffic set to 100% for the deployment!")

## 5. Testing the Deployed Model

In [ ]:
# Load sample data for testing
sample_data = pd.read_csv("../src/model/sample_data.csv")
print("📊 Sample data for testing:")
sample_data.head()

In [ ]:
# Test with standard format
test_input = {
    "data": sample_data.iloc[:3].values.tolist()
}

print("🧪 Testing with standard format...")
result = client.invoke_endpoint(endpoint_name, test_input)

print("📊 Results:")
print(f"Predictions: {result.get('predictions', [])}")
if 'probabilities' in result:
    print(f"Probabilities: {result['probabilities']}")

In [ ]:
# Test with dictionary format
test_input_dict = {
    "inputs": sample_data.iloc[:2].to_dict("list")
}

print("🧪 Testing with dictionary format...")
result_dict = client.invoke_endpoint(endpoint_name, test_input_dict)

print("📊 Results:")
print(f"Predictions: {result_dict.get('predictions', [])}")
if 'probabilities' in result_dict:
    print(f"Probabilities: {result_dict['probabilities']}")

In [ ]:
# Performance test
import time

print("⚡ Performance test - making 10 requests...")
simple_input = {"data": [sample_data.iloc[0].values.tolist()]}

start_time = time.time()
results = []

for i in range(10):
    try:
        result = client.invoke_endpoint(endpoint_name, simple_input)
        results.append(result)
    except Exception as e:
        print(f"Request {i+1} failed: {str(e)}")

end_time = time.time()
duration = end_time - start_time

print(f"✅ {len(results)}/10 requests successful")
print(f"⏱️ Total time: {duration:.2f} seconds")
print(f"📊 Average time per request: {duration/10:.2f} seconds")

## 6. Endpoint Information

In [ ]:
# Get updated endpoint information
endpoints = client.list_endpoints()
for ep in endpoints:
    if ep["name"] == endpoint_name:
        print("📋 Endpoint Information:")
        print(f"  Name: {ep['name']}")
        print(f"  Scoring URI: {ep['scoring_uri']}")
        print(f"  Auth Mode: {ep['auth_mode']}")
        print(f"  Status: {ep['provisioning_state']}")
        break

## 7. Cleanup (Optional)

**⚠️ Warning**: Uncomment and run the cell below only when you're done with the endpoint, as it will delete all resources.

In [ ]:
# Uncomment to delete the endpoint and clean up resources
# print("🧹 Cleaning up resources...")
# client.delete_endpoint(endpoint_name)
# print("✅ Endpoint deleted successfully!")

## Summary

This notebook demonstrated:

1. ✅ **Model Training**: Created and trained a sample RandomForest classifier
2. ✅ **Model Registration**: Registered the model in Azure ML workspace
3. ✅ **Endpoint Creation**: Created a managed online endpoint
4. ✅ **Model Deployment**: Deployed the model with proper scoring script
5. ✅ **Testing**: Tested the endpoint with various input formats
6. ✅ **Performance**: Measured endpoint response times

### Next Steps

- Use the scoring URI in your applications for real-time inference
- Monitor the endpoint performance in Azure ML Studio
- Scale the deployment based on your needs
- Implement A/B testing with multiple deployments
- Add authentication and security measures for production use

### Resources

- [Azure ML Documentation](https://docs.microsoft.com/en-us/azure/machine-learning/)
- [Azure ML SDK v2 Reference](https://docs.microsoft.com/en-us/python/api/overview/azure/ai-ml-readme)
- [Managed Online Endpoints](https://docs.microsoft.com/en-us/azure/machine-learning/concept-endpoints)